# 待解問題
- [X] 自動下拉距離
- [X] 內文格式不同的狀況
- [X] 浮文問題
- [X] 發文日期
- [X] 寫進excel
- [ ] chrome版本bug
- [ ] 查看更多的點擊狀況

In [1]:
import re
import pandas as pd
import time
from datetime import datetime, timedelta
from collections import defaultdict
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import ElementNotVisibleException, StaleElementReferenceException

In [2]:
def last_timestamp():
    
    date_class = ".b6zbclly.myohyog2.l9j0dhe7.aenfhxwr.l94mrbxd.ihxqhq3m.nc684nl6.t5a262vz.sdhka5h4"
    
    try:
        wait = WebDriverWait(browser, 1)
        wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, date_class)))
        timestamp_list = browser.find_elements_by_css_selector(date_class)
    except(StaleElementReferenceException):
        print('stale element reference: element is not attached to the page document')
        
    timestamp = []
    
    for ts in timestamp_list:
        if ts.text != '':
            timestamp.append(ts.text)

    last_timestamp = timestamp[-1]
    
    return last_timestamp

In [3]:
def dateformatter(timestamp):

    if re.search(r'分鐘|小時|2020年12月19日', timestamp):
        mmdd = datetime.today().strftime('%m/%d')
    elif re.search(r'昨天', timestamp):
        mmdd = (datetime.today() - timedelta(1)).strftime('%m/%d')
    elif re.match(r'[\d]{1,2}月[\d]{1,2}日', timestamp):
        mmdd = re.sub(r'日', '', re.sub(r'月', '/', re.search(r'[\d]{1,2}月[\d]{1,2}日', timestamp).group()))
    else:
        print(f'error timestamp: {timestamp}')
        
    yyyymmdd = datetime.today().strftime('%Y/') + mmdd
    date = datetime.strptime(yyyymmdd, "%Y/%m/%d")

    return yyyymmdd, date

In [13]:
def date_of_post(start_date):
    
    last_height = browser.execute_script("return document.body.scrollHeight")
    time.sleep(2)
    
    # 最尾端時間戳記
    last_ts = last_timestamp()
    
    # 日期判斷
    yyyymmdd, date = dateformatter(last_ts)
    
    # 截止日判斷
    if date >= start_date:
        return True
    else:
        print(date)
        new_height = last_height + 1000
        browser.execute_script(f"window.scrollTo(0, {new_height});")
        last_ts = last_timestamp()
        
        if date > start_date:
            return True
        else:
            print(date)
            return False

In [34]:
def see_more():
    
    click_string = "div[class='oajrlxb2 g5ia77u1 qu0x051f esr5mh6w e9989ue4 r7d6kgcz rq0escxv nhd2j8a9 nc684nl6 p7hjln8o kvgmc6g5 cxmmr5t8 oygrvhab hcukyx3x jb3vyjys rz4wbd8a qt6c0cv9 a8nywdso i1ao9s8h esuyzwwr f1sip0of lzcic4wl oo9gr5id gpro0wi8 lrazzd5p'][role='button']"

    try:
        wait = WebDriverWait(browser, 2)
        wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, click_string)))
        
        wait = WebDriverWait(browser, 2)
        wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, click_string)))
        
        read_button = browser.find_element_by_css_selector(click_string)
        browser.implicitly_wait(1)
        if read_button:
            browser.implicitly_wait(2)
            read_button.click()
    except ElementNotVisibleException:
        print("Exception：element not visible")
    except Exception as e:
        print(Exception, e)    
    
#     try:
#         read_button = browser.find_element_by_css_selector(click_string)
#         browser.implicitly_wait(1)
#         if read_button:
#             read_button.click()
#     except ElementNotVisibleException:
#         print("Exception：element not visible")
#     except Exception as e:
#         print(Exception, e)

In [17]:
# 內容
def get_company_content(company, browser):
    
    all_cont = browser.find_elements_by_css_selector("div[class='rq0escxv l9j0dhe7 du4w35lb']")
    
    for cont in all_cont:
        sens = cont.text.split('\n')
        for index, info in enumerate(sens):
            
            if re.search(r'的真實價值', info):
                sub_content = re.split(',|、|的真實價值',info)
                for i, sub in enumerate(sub_content):
                    code = re.search(r'\d{4}', sub)                        
                    if code:
                        code = code.group()
                        name = re.split(r'\(|（', sub)[0].strip()
                        name = re.sub(r'.\n', '', name)
                        company[name]['Code'] = code
                        
                        # get timestamp
                        for ts_index in range(5):
                            if re.search(r'分鐘|小時|昨天|[\d]{1,2}月[\d]{1,2}日', sens[index-(ts_index+1)]):
                                ts = sens[index-(ts_index+1)]

                        yyyymmdd, date = dateformatter(ts)                            
                        #company[name]['Timestamp'] = ts
                        company[name]['Date'] = yyyymmdd

            elif re.search(r'的 Ks =', info):
                index = re.split('的 Ks =', info)
                index_name = index[0].strip()
                index_value = index[1].strip()
                company[index_name]['Ks'] = index_value

            elif re.search(r'的 P / E =', info):
                index = re.split('的 P / E =', info)
                index_name = index[0].strip()
                index_value = index[1].split('=')[-1].strip()
                company[index_name]['P / E'] = index_value

            elif re.search(r'的 FRV =', info):
                index = re.split('的 FRV =', info)
                index_name = index[0].strip()
                index_value = index[1].split('=')[-1].strip()
                company[index_name]['FRV'] = index_value

            elif re.search(r'的 泡沫程度 =', info):
                index = re.split('的 泡沫程度 =', info)
                index_name = index[0].strip()
                index_value = index[1].split('=')[-1].strip()
                company[index_name]['泡沫程度'] = index_value
            
    return company

In [15]:
def main(start_date):

    post = date_of_post(start_date)
    browser.implicitly_wait(2)

    while(post):
        
        last_height = browser.execute_script("return document.body.scrollHeight")
        new_height = last_height + 500
        print(new_height)
        browser.execute_script(f"window.scrollTo(0, {new_height});")
        browser.implicitly_wait(5)

        # 確認目前下拉範圍
        post = date_of_post(start_date)

    if not post:
        print('---------------[To Top]----------------')
        
        last_height = browser.execute_script("return document.body.scrollHeight")
        browser.execute_script(f"window.scrollTo(0, {last_height});")
        time.sleep(1)
        
        browser.execute_script("window.scrollTo(0, 0);")

        check_height = 0
        while(check_height < last_height):
            see_more()
            browser.implicitly_wait(2)
            browser.execute_script(f"window.scrollTo(0, {check_height});")
            
            browser.implicitly_wait(2)            
            get_company_content(company, browser)
            check_height += 500

In [35]:
#browser = webdriver.Chrome(executable_path='.\\webdriver\\chromedriver.exe', chrome_options=opts)
browser = webdriver.Chrome(executable_path='.\\webdriver\\chromedriver.exe')
browser.get('https://www.facebook.com/groups/1189431674567215/')
company = defaultdict(dict)

In [36]:
%%time
start_date = datetime.strptime('20210709', "%Y%m%d")
main(start_date)

2391
5896
9714
15219
19526
2021-07-08 00:00:00
2021-07-08 00:00:00
---------------[To Top]----------------
<class 'Exception'> Message: element click intercepted: Element <div class="oajrlxb2 g5ia77u1 qu0x051f esr5mh6w e9989ue4 r7d6kgcz rq0escxv nhd2j8a9 nc684nl6 p7hjln8o kvgmc6g5 cxmmr5t8 oygrvhab hcukyx3x jb3vyjys rz4wbd8a qt6c0cv9 a8nywdso i1ao9s8h esuyzwwr f1sip0of lzcic4wl oo9gr5id gpro0wi8 lrazzd5p" role="button" tabindex="0">...</div> is not clickable at point (399, 87). Other element would receive the click: <div class="rq0escxv l9j0dhe7 du4w35lb j83agx80 g5gj957u rj1gh0hx buofh1pr hpfvmrgz i1fnvgqd bp9cbjyn owycx6da btwxx1t3 dlv3wnog rl04r1d5">...</div>
  (Session info: chrome=91.0.4472.124)

<class 'Exception'> Message: element click intercepted: Element <div class="oajrlxb2 g5ia77u1 qu0x051f esr5mh6w e9989ue4 r7d6kgcz rq0escxv nhd2j8a9 nc684nl6 p7hjln8o kvgmc6g5 cxmmr5t8 oygrvhab hcukyx3x jb3vyjys rz4wbd8a qt6c0cv9 a8nywdso i1ao9s8h esuyzwwr f1sip0of lzcic4wl oo9gr5id gpro0w

In [37]:
company

defaultdict(dict,
            {'聖輝': {'Code': '5536', 'Date': '2021/07/11'},
             '寶雅': {'Code': '5904',
              'Date': '2021/07/11',
              'Ks': '0.100',
              'P / E': '10.00',
              'FRV': '258.90',
              '泡沫程度': '2.055'},
             '日月光': {'Code': '3711',
              'Date': '2021/07/11',
              'Ks': '0.071',
              'P / E': '14.08',
              'FRV': '124.19',
              '泡沫程度': '0.894'},
             '由田': {'Code': '3455',
              'Date': '2021/07/11',
              'Ks': '0.052',
              'P / E': '19.23',
              'FRV': '80.77',
              '泡沫程度': '0.756'},
             '王品': {'Code': '2727',
              'Date': '2021/07/11',
              'Ks': '0.046',
              'P / E': '21.74',
              'FRV': '161.75',
              '泡沫程度': '1.036'},
             '智冠': {'Code': '5478',
              'Date': '2021/07/11',
              'Ks': '0.052',
              'P / E': '19.23',
      

In [261]:
def to_file(company):
    
    df_list = []
    for com in company.keys():
        if len(company[com].keys()) == 6:
            company[com]['Name'] = com
            df_list.append(company[com])

    df = pd.DataFrame(df_list, columns=['Name', 'Code', 'Ks', 'P / E', 'FRV', '泡沫程度'])
    current_date = datetime.now().strftime('%Y%m%d')
    df.to_csv(f'./{current_date}.csv', index=False, encoding="utf-8-sig")

In [27]:
to_file(company)

In [8]:
browser.close()

In [186]:
s = '昨天上午10:10'
if re.search(r'昨天', s):
    mmdd = (datetime.today() - timedelta(1)).strftime('%m/%d')
sr = datetime.today().strftime('%Y/') + mmdd
sr

'2021/07/10'

In [69]:
my_date = datetime.strptime('20210401', "%Y%m%d")
my_date

datetime.datetime(2021, 4, 1, 0, 0)

In [48]:
datetime.today()

datetime.datetime(2021, 7, 10, 16, 35, 46, 551899)

In [ ]:
opts = Options() # 啟動無頭模式
opts.add_argument("no-sandbox")
opts.add_argument('--disable-gpu')
opts.add_argument("--window-size=800,600")
opts.add_argument("--disable-dev-shm-usage")